* Content Moderatorリソースを作る

# Moderating Personal Information
* https://westus.dev.cognitive.microsoft.com/docs/services/57cf753a3f9b070c105bd2c1/operations/57cf753a3f9b070868a1f66c

In [1]:
subscription_key = 'xxxx'

In [2]:
import urllib.parse

text_headers = {
    'Ocp-Apim-Subscription-Key': subscription_key,
    'Content-Type':'text/plain'
}

text_body = 'Crap! No, it is no longer available, but you can contact me on username@domain.com or 7711611231 so we arrange an external (cheaper) solution.'

# PII:Personal identifiable information
# ClassifyをTrueにすると以下の分類が可能
# Category1 : Sexually explicit
# Category2 : Mature Content
# Category3 : Offensive
text_params = urllib.parse.urlencode({
    'PII':True,
    'classify':True
})

In [3]:
import http.client, json

try:
    conn = http.client.HTTPSConnection('southcentralus.api.cognitive.microsoft.com')
    conn.request('POST', '/contentmoderator/moderate/v1.0/ProcessText/Screen?%s' % text_params, text_body, text_headers)
    response = conn.getresponse()
    jsonData = response.read()
    text_data = json.loads(jsonData)
    print(json.dumps(text_data, indent=2, sort_keys=True))
    conn.close()
except Exception as ex:
    print(ex)

{
  "Classification": {
    "Category1": {
      "Score": 0.0006201675278134644
    },
    "Category2": {
      "Score": 0.22223179042339325
    },
    "Category3": {
      "Score": 0.9879999756813049
    },
    "ReviewRecommended": true
  },
  "Language": "eng",
  "Misrepresentation": null,
  "NormalizedText": "Crap! ,    longer available,  you  contact   username@domain.com  7711611231   arrange  external (cheaper) solution.",
  "OriginalText": "Crap! No, it is no longer available, but you can contact me on username@domain.com or 7711611231 so we arrange an external (cheaper) solution.",
  "PII": {
    "Address": [],
    "Email": [
      {
        "Detected": "username@domain.com",
        "Index": 63,
        "SubType": "Regular",
        "Text": "username@domain.com"
      }
    ],
    "IPA": [],
    "Phone": [
      {
        "CountryCode": "US",
        "Index": 86,
        "Text": "7711611231"
      }
    ],
    "SSN": []
  },
  "Status": {
    "Code": 3000,
    "Description": "

# Moderating Profanity

In [4]:
# Termsに悪い単語が入っているので、置換する
moderated_text = text_data['NormalizedText']

for term in text_data['Terms']:
    old_text = moderated_text[term['Index']:term['Index']+len(term['Term'])]
    print(old_text)
    new_text = '*' * len(term['Term'])
    print(new_text)
    moderated_text = moderated_text.replace(old_text, new_text)
    print(moderated_text)

Crap
****
****! ,    longer available,  you  contact   username@domain.com  7711611231   arrange  external (cheaper) solution.


# Moderating Images

In [5]:
image_evaluate_endpoint = '/contentmoderator/moderate/v1.0/ProcessImage/evaluate?%s'
service_url = 'southcentralus.api.cognitive.microsoft.com'

image_evaluate_headers = {
    'Ocp-Apim-Subscription-Key': subscription_key,
    'Content-Type':'application/json'
}

image_evaluate_body = {
    'DataRepresentation':'URL',
    'Value':'https://wpcdn.us-east-1.vip.tn-cloud.net/www.pittsburghparent.com/content/uploads/data-import/329a3188/background-of-people-smiling-4184-1024x490.jpg'
}


# CacheIDが得られる
image_evaluate_params = urllib.parse.urlencode({
    'CacheImage':True
})

try:
    conn = http.client.HTTPSConnection(service_url)
    conn.request('POST', image_evaluate_endpoint % image_evaluate_params, str(image_evaluate_body), image_evaluate_headers)
    response = conn.getresponse()
    jsonData = response.read()
    data = json.loads(jsonData)
    print(json.dumps(data, indent=2, sort_keys=True))
    conn.close()
except Exception as ex:
    print(ex)

{
  "AdultClassificationScore": 0.0007937020272947848,
  "AdvancedInfo": [
    {
      "Key": "ImageDownloadTimeInMs",
      "Value": "466"
    },
    {
      "Key": "ImageSizeInBytes",
      "Value": "90815"
    }
  ],
  "CacheID": "b1e9002a-1939-4d88-b904-09a4b0fda03c_637970080676357088",
  "IsImageAdultClassified": false,
  "IsImageRacyClassified": false,
  "RacyClassificationScore": 0.001874931447673589,
  "Result": false,
  "Status": {
    "Code": 3000,
    "Description": "OK",
    "Exception": null
  },
  "TrackingId": "138826e1-e61d-418d-84b1-46ed674ff820"
}


In [6]:
# 顔検出も可能
image_findface_endpoint = '/contentmoderator/moderate/v1.0/ProcessImage/findfaces?%s'
try:
    conn = http.client.HTTPSConnection(service_url)
    conn.request('POST', image_findface_endpoint % image_evaluate_params, str(image_evaluate_body), image_evaluate_headers)
    response = conn.getresponse()
    jsonData = response.read()
    data = json.loads(jsonData)
    print(json.dumps(data, indent=2, sort_keys=True))
    conn.close()
except Exception as ex:
    print(ex)

{
  "AdvancedInfo": [
    {
      "Key": "ImageDownloadTimeInMs",
      "Value": "32"
    },
    {
      "Key": "ImageSizeInBytes",
      "Value": "90815"
    }
  ],
  "CacheID": "bb69845b-e6dc-401e-9ae2-33144c1dbbac_637970080688891991",
  "Count": 13,
  "Faces": [
    {
      "Bottom": 316,
      "Left": 445,
      "Right": 521,
      "Top": 240
    },
    {
      "Bottom": 170,
      "Left": 776,
      "Right": 850,
      "Top": 96
    },
    {
      "Bottom": 323,
      "Left": 673,
      "Right": 746,
      "Top": 250
    },
    {
      "Bottom": 350,
      "Left": 829,
      "Right": 901,
      "Top": 278
    },
    {
      "Bottom": 146,
      "Left": 321,
      "Right": 392,
      "Top": 75
    },
    {
      "Bottom": 192,
      "Left": 149,
      "Right": 219,
      "Top": 122
    },
    {
      "Bottom": 211,
      "Left": 503,
      "Right": 573,
      "Top": 141
    },
    {
      "Bottom": 246,
      "Left": 356,
      "Right": 425,
      "Top": 177
    },
    {
      "Bot